In [13]:
import pandas as pd
import numpy as np
import pycountry

In [14]:
df2=pd.read_csv("example.csv")
df = pd.read_csv("homework.csv", low_memory=False)
pd.set_option("display.max_columns", 146)
pd.set_option("display.max_rows", 10)

The following cell changes the date format to ISO 8601

In [15]:
df["system creation date"] = pd.to_datetime(df["system creation date"], format='%m/%d/%y').dt.strftime('%Y-%m-%d')

The following cell is converting the currency datatype to float and rounding it off to cents

In [16]:
def process_money(money):
    if type(money) == float:
        return money
    elif type(money) == int:
        return float(money)
    elif type(money) == str:
        return float(money.strip().replace("$", "").replace(",", "").strip())

for i in df.columns:
    if "$" in i:
        df[i]=df.apply(lambda row: process_money(row[i]), axis=1)
        df[i]=df[i].apply(lambda x: round(x, 2) if pd.notnull(x) else x)

There were columns that contained dimensions in cubic feet. In the following cell, they are converted to cubic inches as required

In [17]:
def cubic_feet_to_inches(cubic_feet):
    inches = cubic_feet * 12**3
    return inches

for col in df.columns:
    if ("feet" in col):
        df[col] = df[col].apply(cubic_feet_to_inches)
        df.rename(columns={col: col.replace("feet", "inches" )}, inplace=True)

homework.csv contained column names that included "carton". To map the column names according to example.csv, the following cell was created

In [18]:
for m in df.columns:
    for i in range(1,4):
        for k in ["width","length",'height','weight','volume']:
            j=str(i) + " " + k
            if j in m:
                #print("boxes__"+str(i-1)+"__"+k)
                df.rename(columns={m: "boxes__"+str(i-1)+"__"+k }, inplace=True)

df.rename(columns={"carton2volumecubicinches": "boxes__"+str(2)+"__"+'volume' }, inplace=True)#we had an exceptional case, that wasnt in a generic form

the following cell is treating the country names and converting them to Alpha_3, as it was shown in the example.csv file

In [19]:
def Country_to_alpha_3(x):
    if not pycountry.countries.get(name=x):
        return np.nan
    return pycountry.countries.get(name=x).alpha_3
#in this case we had an exception of "Philippines" and "Viet Nam" as their names were misspelled in the original document
df["country of origin"]=df["country of origin"].replace(["Phillipines", 'Vietnam'],["Philippines","Viet Nam"])
df["country of origin"]=df["country of origin"].fillna("temp")##temp value, to be replaced afterwards
df["country of origin"].apply(lambda x: Country_to_alpha_3(x))
df.rename(columns={"country of origin": "product__country_of_origin__alpha_3"}, inplace=True)

In [20]:
#replacing the word "item" by "product" as shown in the example.csv
df.columns = [c.replace("item", "product") for c in list(df.columns)]

df.rename(columns={"description": "product__description"},inplace = True)

df.rename(columns={"brand": "product__brand__name"},inplace = True)

df.rename(columns={"item category": "product__product_class__name"},inplace = True)

#in the example.csv file, some of the columns had the word "Attrib" in the beginning, i according to my understanding, added this word to some column names
l=df.columns.get_loc("bulb 1 count")
h=df.columns.get_loc("furniture weight capacity (pounds)")

for i in range(l,h+1):
    df.rename(columns={df.columns[i]: "attrib "+df.columns[i]},inplace = True)

# adding "__" between the words in column names to match the "example.csv" format    
for m in df.columns:
    k= m.split(" ")
    if(len(k)>1):
        df.rename(columns={m: "__".join(k)},inplace = True)

the following cell is converting UPC type to string, as required.

In [21]:
df['upc']=df['upc'].astype(str)

In [22]:
for i in df2.columns:
    print (i)

manufacturer_sku
ean13
weight
length
width
height
prop_65
cost_price
min_price
made_to_order
product__product_class__name
product__brand__name
product__title
product__description
product__bullets__0
product__bullets__1
product__bullets__2
product__bullets__3
product__bullets__4
product__bullets__5
product__bullets__6
product__configuration__codes
product__multipack_quantity
product__country_of_origin__alpha_3
product__parent_sku
attrib__arm_height
attrib__assembly_required
attrib__back_material
attrib__blade_finish
attrib__bulb_included
attrib__bulb_type
attrib__color
attrib__cord_length
attrib__design_id
attrib__designer
attrib__distressed_finish
attrib__fill
attrib__finish
attrib__frame_color
attrib__hardwire
attrib__kit
attrib__leg_color
attrib__leg_finish
attrib__material
attrib__number_bulbs
attrib__orientation
attrib__outdoor_safe
attrib__pile_height
attrib__seat_depth
attrib__seat_height
attrib__seat_width
attrib__shade
attrib__size
attrib__switch_type
attrib__ul_certified
attri

In [23]:
for i in df.columns:
    print (i)

product__number
upc
new__product
system__creation__date
min__order__qty
sales__uom
wholesale__($)
map__($)
msrp__($)
product__description
long__description
product__brand__name
product__category
product__type
outdoor
product__width__(inches)
product__depth__(inches)
product__height__(inches)
product__diameter__(inches)
product__weight__(pounds)
multi-piece__dimension__1__(inches)
multi-piece__dimension__2__(inches)
multi-piece__dimension__3__(inches)
multi-piece__dimension__4__(inches)
product__materials
primary__color__family
product__finish
product__finish__1
product__finish__2
product__finish__3
primary__image__filename
url__primary__image
url__alternate__image__1
url__alternate__image__2
url__alternate__image__3
url__alternate__image__4
url__alternate__image__5
url__alternate__image__6
url__alternate__image__7
url__alternate__image__8
url__room__setting__image__1
url__room__setting__image__2
url__room__setting__image__3
url__drawing
url__interactive__360__image
url__animated__gif
u

All the units of weight were found to be in pounds, hence there was no need for any conversion.

no alternative names were found for some columns in "homework.csv" file, therefore these names were left as they were.

finally, converting the processed dataframe to another CSV file

In [12]:
df.to_csv('formatted.csv')